In [58]:
import pulp
import pandas as pd

# Load the next day's predictions from CSV
df = pd.read_csv("daily_pred.csv")

# Extract necessary information from CSV
tariff_buy = df['pred_tariff']  # Buying tariff
tariff_sell = tariff_buy - 1  # Selling tariff is 1 unit less than buying tariff
solar_output = df['pred_solar_output_kW']  # Predicted solar generation
consumption = df['pred_hourly_consumption']  # Predicted energy consumption
initial_battery_level = df['pred_batteryleft'][0]  # Starting battery level for the day

# Device power ratings (in kW)
washing_machine_power = 1.5  # 1.5 kW for 1 hour
dishwasher_power = 1.2       # 1.2 kW for 2 hours
pump_power = 1               # 1 kW for 1 hour

# Battery parameters
battery_capacity = 15  # Maximum 15 kWh capacity
charge_discharge_rate = 3  # Max 3 kW charging/discharging rate

# Create the optimization model
model = pulp.LpProblem("HEMS_Optimization", pulp.LpMinimize)

# Define decision variables as binary
x_wm = [pulp.LpVariable(f"x_wm_{t}", cat="Binary") for t in range(24)]  # Washing Machine
x_dw = [pulp.LpVariable(f"x_dw_{t}", cat="Binary") for t in range(24)]  # Dishwasher
x_pump = [pulp.LpVariable(f"x_pump_{t}", cat="Binary") for t in range(24)]  # Pump

# Battery variables
battery_charge = [pulp.LpVariable(f"battery_charge_{t}", lowBound=0, upBound=charge_discharge_rate) for t in range(24)]
battery_discharge = [pulp.LpVariable(f"battery_discharge_{t}", lowBound=0, upBound=charge_discharge_rate) for t in range(24)]
battery_level = [pulp.LpVariable(f"battery_level_{t}", lowBound=0, upBound=battery_capacity) for t in range(24)]
grid_power_buy = [pulp.LpVariable(f"grid_power_buy_{t}", lowBound=0) for t in range(24)]
grid_power_sell = [pulp.LpVariable(f"grid_power_sell_{t}", lowBound=0) for t in range(24)]

# Objective function: Minimize total cost from grid (buying) while maximizing profit from selling
model += pulp.lpSum(
    tariff_buy[t] * grid_power_buy[t] - tariff_sell[t] * grid_power_sell[t] 
    for t in range(24)
)

# Constraints

# Power balance constraint for each hour
for t in range(24):
    model += (washing_machine_power * x_wm[t] + 
              dishwasher_power * x_dw[t] + 
              pump_power * x_pump[t] + consumption[t] <= 
              grid_power_buy[t] - grid_power_sell[t] + battery_discharge[t] + solar_output[t] - battery_charge[t])

# Battery state of charge constraints
for t in range(24):
    if t == 0:
        model += battery_level[t] == initial_battery_level + battery_charge[t] - battery_discharge[t]
    else:
        model += battery_level[t] == battery_level[t-1] + battery_charge[t] - battery_discharge[t]

# Device operation constraints

# Washing machine: exactly 1 hour of operation (1 interval)
model += pulp.lpSum(x_wm) == 1  # Ensures it only runs for 1 hour

# Dishwasher: exactly 2 consecutive hours of operation
model += pulp.lpSum(x_dw) == 2  # Total of 2 hours
for t in range(23):  # Iterate up to 23 to avoid index errors on t+1
    # Ensures that if it starts at t, it continues for the next hour
    model += x_dw[t] + x_dw[t+1] <= 2
    model += x_dw[t] - x_dw[t+1] <= 0  # Enforces consecutive blocks of [1,1] or [0,0]

# Pump: exactly 1 hour of operation (1 interval)
model += pulp.lpSum(x_pump) == 1  # Ensures it only runs for 1 hour

# Solve the optimization model
solver = pulp.PULP_CBC_CMD(msg=True)  # Default tolerance
model.solve(solver)

# Print results
print("Status:", pulp.LpStatus[model.status])
print("Total Grid Cost:", pulp.value(model.objective))

# Retrieve and round results for binary device schedules
device_results = {
    'washing_machine': [round(pulp.value(x_wm[t])) for t in range(24)],
    'dishwasher': [round(pulp.value(x_dw[t])) for t in range(24)],
    'pump': [round(pulp.value(x_pump[t])) for t in range(24)]
}

# Check if values are binary
for device, values in device_results.items():
    if any(value not in [0, 1] for value in values):
        print(f"Warning: {device} schedule contains non-binary values after rounding:", values)

# Display optimized schedule for devices and battery
results = pd.DataFrame({
    'hour': range(24),
    'grid_power_buy': [max(0, pulp.value(grid_power_buy[t])) for t in range(24)],  # Ensures non-negativity
    'grid_power_sell': [max(0, pulp.value(grid_power_sell[t])) for t in range(24)],  # Ensures non-negativity
    'battery_level': [pulp.value(battery_level[t]) for t in range(24)],
    'battery_charge': [pulp.value(battery_charge[t]) for t in range(24)],
    'battery_discharge': [pulp.value(battery_discharge[t]) for t in range(24)],
    'washing_machine': device_results['washing_machine'],
    'dishwasher': device_results['dishwasher'],
    'pump': device_results['pump']
})
print(results)


Status: Optimal
Total Grid Cost: -28.874007582497867
    hour  grid_power_buy  grid_power_sell  battery_level  battery_charge  \
0      0        4.957526         0.000000       9.396005        3.000000   
1      1        3.662052         0.000000      10.990073        1.594069   
2      2        0.000000         0.000000       9.000000        0.000000   
3      3        5.099734         0.000000      12.000000        3.000000   
4      4        0.000000         0.561115       9.000000        0.000000   
5      5        0.032319         0.000000       6.000000        0.000000   
6      6        0.040689         0.000000       3.000000        0.000000   
7      7        6.045960         0.000000       6.000000        3.000000   
8      8        5.658555         0.000000       9.000000        3.000000   
9      9        4.596672         0.000000      12.000000        3.000000   
10    10        0.000000         3.755148       9.000000        0.000000   
11    11        0.000000         7.